## 필요 라이브러리 패키지 불러오기

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, AdamW

import torchvision.transforms as T
from torchvision.transforms import Compose, RandomHorizontalFlip, Normalize, RandomRotation

from PIL import Image, ImageOps

## 이미지 Crop & Resizing

In [2]:
# 가로/세로 비율 구하는 함수 
def aspect_ratio(w, h):
    return abs(w / h)

In [3]:
def custom_crop_and_resize(img, diff_thresh = 0.33, 
                           pad_color=(114,114,114)): # 1:1에서 dev가 이 값 초과면 크롭(내용 보존↑면 0.33~0.35)
    """
    - |w/h - 1| > diff_thresh면 중앙 '크롭', 아니면 '패딩'으로 정사각
    - 그 다음에만 target_size로 리사이즈 (비율 왜곡 없음)
    """
    # EXIF 방향 보정 + RGB
    img = ImageOps.exif_transpose(img).convert("RGB") # EXIF Orientation을 실제 픽셀에 반영해 올바른 방향으로 정규화

    width, height = img.size
    dev = abs(width / height - 1.0)

    # 가로 세로 비율 차이가 크면 크롭을 적용합니다.
    if dev > diff_thresh:
        side = min(width, height)
        left = (width - side) // 2
        top  = (height - side) // 2
        img = img.crop((left, top, left + side, top + side))
    # 비율 차이가 작으면 패딩을 적용하여 정사각형으로 만듭니다.
    else:
        max_side = max(width, height)
        new_img = Image.new('RGB', (max_side, max_side), pad_color)
        new_img.paste(img, ((max_side - width) // 2, (max_side - height) // 2))
        img = new_img
    
    return img

## 훈련용 / 테스트 데이터셋

In [4]:
df = pd.read_csv('../../csv/cleaned_total.csv')

# 가격 이상치 정규화(한쪽으로 치우침 완하)
df['price'] = np.log1p(df['price'])

train_df = df[['price','image_path']].sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

In [5]:
print(len(train_df), len(test_df)) # 훈련용 데이터 개수 / 테스트 데이터 개수

3346 836


## 커스텀 데이터셋

In [6]:
class PirceImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe # CSV 불러온 데이터
        self.transform = transform

    def __len__(self):
        return len(self.df) # 데이터 개수

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(f'../../{row["image_path"]}').convert("RGB")  # 이미지를 RGB 이미지로 읽기
        if self.transform:
            img = self.transform(img) # 이미지 텐서형으로 변환
        label = torch.tensor([row["price"]], dtype=torch.float32) # 정답라벨을 가격데이터로 설정
        return img, label

In [7]:
transforms_train = Compose(
    [ # (530, 690)
        T.Lambda(lambda img: custom_crop_and_resize(img, aspect_ratio(720, 720))), # 이미지들의 평균 w, h 값
        T.ToTensor(),
        T.Resize((224, 224)), # VGG16의 권장된 resizing 값
        RandomRotation(degrees=(-15, 15)), # 과적합 방지위해 이미지를 무작위 회전
        RandomHorizontalFlip(p=0.5), # 과적합 방지위헤 50%로 좌우 대칭 변환
        Normalize(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

transforms_test = Compose(
    [# (530, 690)
        T.Lambda(lambda img: custom_crop_and_resize(img, aspect_ratio(720, 720))), # 이미지들의 평균 w, h 값
        T.ToTensor(),
        T.Resize((224, 224)), # VGG16의 권장된 resizing 값
        Normalize(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

In [8]:
train_dataset = PirceImageDataset(train_df, transforms_train)
test_dataset = PirceImageDataset(test_df, transforms_train)

In [9]:
df.isna().sum()

id                 0
title              0
detail             1
condition       1519
is_completed       0
price              0
location        1005
source             0
model           2713
model_type      2618
log_price          0
image_path         0
dtype: int64

## 학습 준비

In [10]:
# 32개의 이미지와 라벨을 묶어서 가져오기
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
# 모델 가져오기
from torchvision.models.vgg import vgg16

model = vgg16(pretrained=True) # 가중치값 가져오기
model

c:\Users\user\Documents\SecondHanded-Strollers-PredictedPrice\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\Documents\SecondHanded-Strollers-PredictedPrice\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
# 전부 동결하기: 기존 가중치들은 업데이트 안한다
for param in model.parameters():
    param.requires_grad = False 

# 마지막 FC만 회귀용인 1차원으로 교체하고 학습 켜기
in_feature = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_feature, 1)

# 마지막 FC만 학습 켜기(마지막fc만 업데이트)
for param in model.classifier[-1].parameters():
    param.requires_grad = True

## 학습

In [14]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train() # 학습 모드 켜기

# optim = Adam(model.parameters(), lr=1e-3) # 최적화 함수
optim = AdamW(model.parameters(), lr=0.0014432796537068773, weight_decay=0.0002337417039815359) # 최적화 함수 | 가중치 감쇠(정규화 항)
criterion = nn.MSELoss() # 손실함수
epochs = 15

# 최적화 과정 시각화하기
writer = SummaryWriter()
step = 0

for epoch in range(epochs):
    for images, labels in tqdm(train_dataloader): # 배차사이즈 만큼의 이미지와 라벨들
        optim.zero_grad() # 한번 학습마다 최적화 함수인 기울기 초기화
        preds = model(images.to(device)) # 순전파
        loss = criterion(preds, labels.to(device)) # 손실계산

        loss.backward() # 역전파로 각 파라미터의 기울기 계산
        optim.step() # 기울기를 이용해 파라미터 업데이트
        writer.add_scalar("Loss/train", loss.item(), step)
        step += 1
    print("loss : ", loss.item()) # 현재 손실 출력

100%|██████████| 105/105 [01:51<00:00,  1.06s/it]


loss :  3.57826828956604


100%|██████████| 105/105 [01:55<00:00,  1.10s/it]


loss :  1.279308795928955


100%|██████████| 105/105 [01:41<00:00,  1.04it/s]


...

loss :  2.9252026081085205


100%|██████████| 105/105 [01:29<00:00,  1.17it/s]


loss :  1.7407076358795166


100%|██████████| 105/105 [01:23<00:00,  1.26it/s]

loss :  2.946347951889038


In [15]:
from torchmetrics.regression import R2Score, MeanAbsoluteError

# 모델 평가모드로 설정
model.eval()

with torch.no_grad():        
        # 평가 지표 객체들을 초기화합니다. 이 객체들은 기본적으로 CPU에 있습니다.
        mse_metric = nn.MSELoss()
        mae_metric = MeanAbsoluteError().to(device)
        r2_metric = R2Score().to(device)
        
        total_mse_loss = 0.0
        
        for images, labels in tqdm(test_dataloader):
            # 데이터를 GPU로 이동합니다.
            images = images.to(device)
            labels = labels.to(device)
            
            # 순전파를 수행하여 예측값을 얻습니다.
            preds = model(images)
            
            # total_mse_loss값을 계산을 위해 preds와 labels를 업데이트합니다. 
            mse_loss = mse_metric(preds, labels)
            total_mse_loss += mse_loss.item()
            
            mae_metric.update(preds, labels)
            r2_metric.update(preds, labels)
            
        # 검증 데이터셋에 대한 최종 지표들을 계산합니다.
        avg_mse_loss = total_mse_loss / len(test_dataloader)
        final_mae = mae_metric.compute()
        final_r2_score = r2_metric.compute()
        
        # 결과를 Pandas DataFrame으로 만들어 테이블 형태로 출력합니다.
        results = pd.DataFrame({
            'Metric': ['MSE', 'MAE', 'R2'],
            'Value': [avg_mse_loss, final_mae.item(), final_r2_score.item()]
        })
        
        print("\n모델 평가 결과:")
        print(results.to_string(index=False))

100%|██████████| 27/27 [00:24<00:00,  1.11it/s]


모델 평가 결과:
Metric     Value
   MSE  4.254870
   MAE  1.780848
    R2 -3.840781


`uv run tensorboard --logdir=./src/training/runs`

## 모델의 가중치만 저장

In [16]:
torch.save(model.state_dict(), 'models/adamw_with_optuna.pth') # 딥러닝모델 저장 | 모델의 파라미터 값만 저장 